Install dependencies

!pip install transformers datasets accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 7.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.2 MB/s eta 0:00:00:00:0100:01

load dataset

import pandas as pd
from datasets import Dataset

csv_path = "/kaggle/input/dataset-training-chatbot/train_data_chatbot.csv"
df = pd.read_csv(csv_path)

print("Columns found:", df.columns.tolist())

if not {'short_question', 'short_answer'}.issubset(df.columns):
    raise ValueError("CSV cần có 2 cột: 'short_question' và 'short_answer'")

df["text"] = "User: " + df["short_question"].astype(str) + " Bot: " + df["short_answer"].astype(str)
df = df.dropna(subset=["text"])

dataset = Dataset.from_pandas(df[["text"]])
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"Dataset size — Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")

Columns found: ['short_question', 'short_answer', 'tags', 'label']
Dataset size — Train: 42842, Test: 4761

Tokenization

from transformers import AutoTokenizer

model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    tokens = tokenizer(
        batch["text"], 
        truncation=True, 
        padding="max_length", 
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

Load model

from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name)
model.config.pad_token_id = model.config.eos_token_id


tokenizer_config.json: 100%
 614/614 [00:00<00:00, 66.8kB/s]
vocab.json: 
 1.04M/? [00:00<00:00, 24.1MB/s]
merges.txt: 
 456k/? [00:00<00:00, 36.0MB/s]
Map: 100%
 42842/42842 [00:10<00:00, 4386.00 examples/s]
Map: 100%
 4761/4761 [00:01<00:00, 4490.07 examples/s]
config.json: 100%
 641/641 [00:00<00:00, 91.3kB/s]
model.safetensors: 100%
 351M/351M [00:01<00:00, 273MB/s]
generation_config.json: 100%
 124/124 [00:00<00:00, 15.5kB/s]
/tmp/ipykernel_37/372943805.py:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer()

Training arguments

Bắt đầu huấn luyện...

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="no",
    report_to="none",
    fp16=True
)

Trainer setup

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer
)

<!-- print("Bắt đầu huấn luyện...")
trainer.train()
print("Huấn luyện hoàn tất.") -->

[5356/5356 1:17:24, Epoch 1/1]
Step	Training Loss	Validation Loss
100	1.670300	3.117189
200	1.516600	2.977963
300	1.565900	2.926508
400	1.570700	2.887151
500	1.503200	2.855027
600	1.532700	2.841193
700	1.355200	2.817111
800	1.489900	2.797966
900	1.500100	2.782957
1000	1.445900	2.771570
1100	1.354800	2.759558
1200	1.532000	2.749580
1300	1.577000	2.743269
1400	1.519300	2.733122
1500	1.300000	2.726318
1600	1.361000	2.719471
1700	1.414000	2.710235
1800	1.323800	2.706916
1900	1.304600	2.694760
2000	1.357600	2.688698
2100	1.428000	2.684506
2200	1.390800	2.679331
2300	1.331300	2.673726
2400	1.402900	2.667903
2500	1.466000	2.666873
2600	1.396500	2.659858
2700	1.285500	2.655107
2800	1.336200	2.651288
2900	1.479500	2.648543
3000	1.530800	2.644536
3100	1.337000	2.639378
3200	1.358800	2.636982
3300	1.310800	2.633861
3400	1.246600	2.630315
3500	1.269300	2.627488
3600	1.354300	2.624171
3700	1.267900	2.623065
3800	1.558300	2.620484
3900	1.497300	2.617727
4000	1.275500	2.615196
4100	1.433800	2.613961
4200	1.317700	2.610878
4300	1.270600	2.609249
4400	1.395800	2.608568
4500	1.359400	2.606919
4600	1.453300	2.605266
4700	1.354000	2.604426
4800	1.250400	2.602848
4900	1.480900	2.602213
5000	1.406100	2.600456
5100	1.386400	2.599710
5200	1.433500	2.599883
5300	1.526200	2.599429


 Save model

save_dir = "./fine_tuned_dialoGPT"

import os, torch, gc
gc.collect()
torch.cuda.empty_cache()

if os.path.exists(save_dir):
    !rm -rf {save_dir}

trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Model saved to {save_dir}")

Test model

prompt = "Tôi bị đau ngực và khó thở, có sao không?"
inputs = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors="pt")

outputs = model.generate(
    inputs,
    max_length=100,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7,
    top_k=50,
    do_sample=True
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Chatbot test response:")
print(response)